In [1]:
import os
import pandas as pd
import numpy as np
import glob
from PIL import Image
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.utils import load_img, img_to_array
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Flatten, Dense, Dropout
from tensorflow.keras.optimizers import RMSprop
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import confusion_matrix, classification_report
from tensorflow.keras.utils import to_categorical

In [2]:
def createFrame(path, IMG_DIM):
    train_imgs = []
    labels = []

    # Liste des classes (chaque dossier représente une classe)
    directories = os.listdir(path)

    for i, directory in enumerate(directories):
        class_path = os.path.join(path, directory)
        cropped_path = os.path.join(class_path, 'CROPPED')

        # On vérifie que CROPPED existe
        if not os.path.isdir(cropped_path):
            continue

        # Liste des images dans le dossier CROPPED
        img_files = glob.glob(os.path.join(cropped_path, '*'))

        for img_path in img_files:
            try:
                # Vérifie que c’est bien une image (avec PIL)
                with Image.open(img_path) as img:
                    img.verify()

                # Chargement + redimensionnement
                arr = img_to_array(load_img(img_path, target_size=IMG_DIM))
                train_imgs.append(arr)
                labels.append(i)

            except Exception as e:
                print(f"Ignoré : {img_path} -> {e}")

    # Création d’un DataFrame contenant les images + labels
    df = pd.DataFrame(list(zip(train_imgs, labels)))
    df = df.sample(frac=1).reset_index(drop=True)  # Shuffle
    col=["images","class"]
    df.columns=col
    return df


In [3]:
IMG_WIDTH = 128
IMG_HEIGHT = 128
IMG_DIM = (IMG_WIDTH, IMG_HEIGHT, 3)
path="data"
df=createFrame(path,IMG_DIM)

In [4]:
#know the number associated with each class
directories = os.listdir(path)
for i, directory in enumerate(directories):
        class_path = os.path.join(path, directory)
        print(class_path,i)

data\im_Dyskeratotic 0
data\im_Koilocytotic 1
data\im_Metaplastic 2
data\im_Parabasal 3
data\im_Superficial-Intermediate 4


### Densenet

In [5]:
def DenseNet(train_imgs, train_labels, class_no):
    print("--------------- DENSENET ---------------")

    input_shape_densenet = (128, 128, 3)

    densenet_model = tf.keras.applications.DenseNet169(
        include_top=False,
        weights="imagenet",
        input_shape=input_shape_densenet,
        pooling=None
    )

    # On gèle les couches pré-entraînées
    densenet_model.trainable = True
    for layer in densenet_model.layers:
        layer.trainable = False

    # Ajout de couches personnalisées en sortie du modèle
    x = Flatten()(densenet_model.output)
    x = Dense(units=1024, activation='relu')(x)
    x = Dropout(0.2)(x)
    x = Dense(units=128, activation='relu')(x)
    output = Dense(units=class_no, activation='softmax')(x)

    model = Model(inputs=densenet_model.input, outputs=output)

    model.compile(
        optimizer=RMSprop(learning_rate=2e-5),
        loss='categorical_crossentropy',
        metrics=['acc']
    )

    history = model.fit(
        train_imgs, train_labels,
        batch_size=32,
        verbose=1
    )

    print("--------------- FIN DENSENET ---------------")
    return model

In [6]:
target_names = os.listdir(path)
num_classes = len(target_names)

In [7]:
# 3. Séparation train / test (ici 80% / 20%), en stratifiant sur la colonne 'class'
dfTrain, dfTest = train_test_split(
    df,
    test_size=0.2,
    random_state=42,
    stratify=df["class"]
)

In [8]:
# 4. Préparation des images et labels pour l’entraînement
train_imgs = np.array(dfTrain["images"].tolist(), dtype="float32") / 255.0
train_labels_raw = np.array(dfTrain["class"])
encoder = LabelEncoder().fit(train_labels_raw)
train_labels_enc = encoder.transform(train_labels_raw)
train_labels = to_categorical(train_labels_enc, num_classes=num_classes)

In [9]:
# 5. Préparation des images et labels pour le test
test_imgs = np.array(dfTest["images"].tolist(), dtype="float32") / 255.0
test_labels_raw = np.array(dfTest["class"])
test_labels_enc = encoder.transform(test_labels_raw)
test_labels = to_categorical(test_labels_enc, num_classes=num_classes)

In [10]:
# 6. Entraînement du modèle DenseNet
model_dens= DenseNet(train_imgs, train_labels, class_no=num_classes)

--------------- DENSENET ---------------
101/101 ━━━━━━━━━━━━━━━━━━━━ 71s 560ms/step - acc: 0.7947 - loss: 0.5960
--------------- FIN DENSENET ---------------


In [ ]:
#model.save("saved_models/densenet_model.h5")

### Inception

In [130]:
def Inception(train_imgs, train_labels, class_no):
    print("--------------- INCEPTION ---------------")

    # Charger InceptionV3 sans la couche finale, avec poids ImageNet
    pre_trained_model2 = tf.keras.applications.InceptionV3(
        input_shape=(128, 128, 3),
        include_top=False,
        weights='imagenet'
    )

    # Geler les couches de base
    for layer in pre_trained_model2.layers:
        layer.trainable = False

    # Ajouter des couches personnalisées en tête
    x = Flatten()(pre_trained_model2.output)
    x = Dense(1028, activation='relu')(x)
    x = Dropout(0.2)(x)
    x = Dense(64, activation='relu')(x)
    output = Dense(class_no, activation='softmax')(x)

    # Construire le modèle complet
    model = Model(inputs=pre_trained_model2.input, outputs=output)

    # Compiler le modèle
    model.compile(
        optimizer=RMSprop(learning_rate=2e-5),
        loss='categorical_crossentropy',
        metrics=['acc']
    )

    # Entraîner le modèle
    history = model.fit(
        x=train_imgs, y=train_labels,
        batch_size=32,
        verbose=0
    )

    print("--------------- FIN INCEPTION ---------------")
    return model

In [131]:
model_inception= Inception(train_imgs, train_labels, class_no=num_classes)

--------------- INCEPTION ---------------
--------------- FIN INCEPTION ---------------


In [ ]:
#model_inception.save("saved_models/inception_model.h5")

### Xception

In [133]:
def Xception(train_imgs, train_labels, class_no):
    print("--------------- XCEPTION ---------------")

    pre_trained_model = tf.keras.applications.Xception(
        input_shape=(128, 128, 3),
        include_top=False,
        weights="imagenet"
    )

    # Geler les poids des couches convolutionnelles pré-entraînées
    for layer in pre_trained_model.layers:
        layer.trainable = False

    # Ajouter des couches personnalisées en tête du modèle
    x = Flatten()(pre_trained_model.output)
    x = Dense(256, activation='relu')(x)
    x = Dropout(0.2)(x)
    x = Dense(32, activation='relu')(x)
    output = Dense(class_no, activation='softmax')(x)

    # Construire le modèle final
    model = Model(inputs=pre_trained_model.input, outputs=output)

    # Compiler le modèle
    model.compile(
        optimizer=RMSprop(learning_rate=2e-5),
        loss='categorical_crossentropy',
        metrics=['acc']
    )

    # Entraîner le modèle
    history = model.fit(
        x=train_imgs,
        y=train_labels,
        batch_size=32,
        verbose=0
    )
    print("--------------- FIN XCEPTION ---------------")
    return model

In [134]:
model_xception= Xception(train_imgs, train_labels, class_no=num_classes)

--------------- XCEPTION ---------------
--------------- FIN XCEPTION ---------------


In [ ]:
#model_xception.save("saved_models/xception_model.h5")

c:\Users\masta\anaconda3\envs\DeepLearning\lib\site-packages\keras\src\engine\training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


### Ensemble

In [136]:
# Ensemble basé sur le rang flou :
def getScore(model, test_imgs) :
  res = model.predict (test_imgs)
  return res

In [137]:
def generateRank1(score, class_no) :
    rank = np.zeros ([class_no, 1])
    scores = score
    for i in range(class_no):
        rank[i] =1- np.exp(-((scores[i]-1) ** 2)/2.0)
    return rank

In [138]:
def generateRank2(score, class_no) :
  rank = np.zeros([class_no, 1])
  scores = score
  for i in range(class_no):
    rank[i] = 1 - np.tanh(((scores[i]-1)**2)/2)
  return rank

In [139]:
def doFusion(res1,res2,res3,label,class_no):
  cnt = 0
  id = []
  for i in range(len(res1)):
      rank1 = generateRank1(res1[i],class_no)*generateRank2(res1[i],class_no)
      rank2 = generateRank1(res2[i],class_no)*generateRank2(res2[i],class_no)
      rank3 = generateRank1(res3[i],class_no)*generateRank2(res3[i],class_no)
      rankSum = rank1 + rank2 + rank3
      rankSum = np.array(rankSum)
      scoreSum = 1 - (res1[i] + res2[i] + res3[i])/3
      scoreSum = np.array(scoreSum)
      
      fusedScore = (rankSum.T)*scoreSum
      cls = np.argmin(rankSum)
      if cls<class_no and label[i][cls]== 1:
          cnt += 1
      id.append(cls)
  print(cnt/len(res1))
  return id

In [147]:
res1 = model_dens.predict(test_imgs)
res2 = model_inception.predict(test_imgs) 
res3 = model_xception.predict(test_imgs)
predictedClass = doFusion(res1,res2,res3,test_labels,class_no=num_classes)
y_true = np.argmax(test_labels, axis=-1)


26/26 [==============================] - 19s 746ms/step
0.8695652173913043


array([0.04353547, 0.2752686 , 0.6589483 , 0.0166451 , 0.00560255],
      dtype=float32)

In [ ]:
print("\nClassification Report pour Ensemble :")
print(classification_report(y_true, predictedClass,target_names=target_names))


Classification Report pour Ensemble :
                             precision    recall  f1-score   support

            im_Dyskeratotic       0.85      0.93      0.89       162
            im_Koilocytotic       0.79      0.76      0.77       164
             im_Metaplastic       0.86      0.78      0.82       158
               im_Parabasal       0.92      0.91      0.91       156
im_Superficial-Intermediate       0.93      0.96      0.95       165

                   accuracy                           0.87       805
                  macro avg       0.87      0.87      0.87       805
               weighted avg       0.87      0.87      0.87       805



In [142]:
y_pred_prob = model_xception.predict(test_imgs)

# 2. Passage aux classes prédictes
y_pred = np.argmax(y_pred_prob, axis=-1)

# 5. Classification report (précision, rappel, f1-score)
#    Si tu as la liste des noms de classes dans target_names, tu peux l’y passer
print("\nClassification Report pour XCEPTION :")
print(classification_report(y_true, y_pred,target_names=target_names  # liste de tes classes, ex. ['chien','chat',...]
))

26/26 [==============================] - 29s 1s/step

Classification Report pour XCEPTION :
                             precision    recall  f1-score   support

            im_Dyskeratotic       0.83      0.91      0.86       162
            im_Koilocytotic       0.79      0.71      0.75       164
             im_Metaplastic       0.75      0.68      0.72       158
               im_Parabasal       0.82      0.83      0.83       156
im_Superficial-Intermediate       0.84      0.90      0.87       165

                   accuracy                           0.81       805
                  macro avg       0.81      0.81      0.81       805
               weighted avg       0.81      0.81      0.81       805



In [143]:
y_pred_prob = model_inception.predict(test_imgs)

# 2. Passage aux classes prédictes
y_pred = np.argmax(y_pred_prob, axis=-1)



# 5. Classification report (précision, rappel, f1-score)
#    Si tu as la liste des noms de classes dans target_names, tu peux l’y passer
print("\nClassification Report pour INCEPTION:")
print(classification_report(y_true, y_pred,target_names=target_names  # liste de tes classes, ex. ['chien','chat',...]
))

26/26 [==============================] - 25s 927ms/step

Classification Report pour INCEPTION:
                             precision    recall  f1-score   support

            im_Dyskeratotic       0.73      0.80      0.77       162
            im_Koilocytotic       0.65      0.57      0.61       164
             im_Metaplastic       0.62      0.68      0.65       158
               im_Parabasal       0.86      0.79      0.82       156
im_Superficial-Intermediate       0.90      0.91      0.90       165

                   accuracy                           0.75       805
                  macro avg       0.75      0.75      0.75       805
               weighted avg       0.75      0.75      0.75       805



In [144]:
y_pred_prob = model_dens.predict(test_imgs)

# 2. Passage aux classes prédictes
y_pred = np.argmax(y_pred_prob, axis=-1)

# 3. Récupérer les vraies classes


# 5. Classification report (précision, rappel, f1-score)
#    Si tu as la liste des noms de classes dans target_names, tu peux l’y passer
print("\nClassification Report pour DENSENET :")
print(classification_report(y_true, y_pred,target_names=target_names  # liste de tes classes, ex. ['chien','chat',...]
))

26/26 [==============================] - 41s 2s/step

Classification Report pour DENSENET :
                             precision    recall  f1-score   support

            im_Dyskeratotic       0.86      0.92      0.89       162
            im_Koilocytotic       0.75      0.82      0.78       164
             im_Metaplastic       0.90      0.78      0.83       158
               im_Parabasal       0.90      0.93      0.91       156
im_Superficial-Intermediate       0.99      0.91      0.95       165

                   accuracy                           0.87       805
                  macro avg       0.88      0.87      0.87       805
               weighted avg       0.88      0.87      0.87       805

